#### Build a Classifier with Pytorch
Now that our data has been cleaned and preprocess, we can build a model. 
First, we will split the data into training and testing set.

In [2]:
# import necessary packagaes/libraries
from sklearn.model_selection import train_test_split
import shutil
import os

In [3]:
# set working directory
project_folder = r'C:\Users\Vy\Documents\Lewisville_Lake_Fish _Classification'

# fish folders 
base = os.path.join(project_folder, 'Data')

In [4]:
def split_dataset(base_dir, output_dir=base, test_size=0.2): # file structure Data -> train, test, "the fish species"
    for species in os.listdir(base_dir):
        files = os.listdir(os.path.join(base_dir, species))
        train_files, test_files = train_test_split(files, test_size=test_size, random_state=42)
        
        for split, split_files in zip(['train', 'test'], [train_files, test_files]): 
            split_dir = os.path.join(output_dir, split, species) # creates directory for current split and species
            os.makedirs(split_dir, exist_ok=True)
            for file in split_files:
                src = os.path.join(base_dir, species, file) # source
                dst = os.path.join(split_dir, file) # destination
                shutil.copy(src, dst) # copies files from source to destination
                
# split
split_dataset(base)

In [5]:
# !pip install torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [6]:
# define transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# load datasets
train_dataset = datasets.ImageFolder(r'C:\Users\Vy\Documents\Lewisville_Lake_Fish _Classification\Data\train', transform=transform)
test_dataset = datasets.ImageFolder(r'C:\Users\Vy\Documents\Lewisville_Lake_Fish _Classification\Data\test', transform=transform)

# create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [7]:
images, labels = next(iter(train_loader))
print(images.shape)   # e.g. torch.Size([32, 3, 128, 128]), batch size, rgb channels, image size, respectively
print(labels[:5])     # e.g. tensor([0, 2, 1, 0, 3]), labels for classes
print(train_dataset.classes)

torch.Size([32, 3, 128, 128])
tensor([1, 3, 1, 3, 1])
['Guadalupe_bass', 'Largemouth_bass', 'Smallmouth_bass', 'Spotted_bass']


In [8]:
import torch.nn as nn
import torch.nn.functional as F

class FishClassifier(nn.Module):
    def __init__(self, num_classes):
        super(FishClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # -> 64x64
        x = self.pool(F.relu(self.conv2(x)))  # -> 32x32
        x = x.view(-1, 32 * 32 * 32)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [9]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FishClassifier(num_classes=len(train_dataset.classes)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 17.5068
Epoch 2, Loss: 8.6008
Epoch 3, Loss: 7.8628
Epoch 4, Loss: 7.6472
Epoch 5, Loss: 7.1556
Epoch 6, Loss: 6.3986
Epoch 7, Loss: 4.9108
Epoch 8, Loss: 4.1091
Epoch 9, Loss: 3.0173
Epoch 10, Loss: 2.2380


In [10]:
model.eval()
correct = total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {100 * correct / total:.2f}%")

Validation Accuracy: 40.00%


In [21]:
from sklearn.metrics import classification_report, confusion_matrix

model.eval()  # set model to eval mode

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# generate classification report
print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))

# confusion matrix
print(confusion_matrix(all_labels, all_preds))

                 precision    recall  f1-score   support

 Guadalupe_bass       0.33      0.67      0.44         9
Largemouth_bass       0.50      0.50      0.50        12
Smallmouth_bass       0.44      0.31      0.36        13
   Spotted_bass       0.33      0.18      0.24        11

       accuracy                           0.40        45
      macro avg       0.40      0.41      0.39        45
   weighted avg       0.41      0.40      0.38        45

[[6 2 1 0]
 [2 6 2 2]
 [4 3 4 2]
 [6 1 2 2]]
